In [ ]:
# prepare data
import os
import glob
import json
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split


paths = list(glob.iglob('data/IJCAI_2019_AAAC_train/*/*'))
for i, p in enumerate(paths):
#     with open(p, 'rb') as f:
#         image_bytes = f.read()
#         image_bytes = exif_remove(image_bytes)
#     iof = BytesIO()
#     iof.write(image_bytes)
    if p.endswith(('gif', )):
        status[i] = False
    image = Image.open(p)
    x, y = image.size
    if x < 30 or y < 30:
        status[i] = False
    if image.format == "GIF":
        status[i] = False
    if image.mode == 'RGBA':
        status[i] = False
    if image.mode != "RGB":
        status[i] = False
status = [True] * len(paths)
paths = np.array(paths)
paths = paths[status]
labels = [int(p.split('/')[-2]) for p in paths]
path_train, path_test, label_train, label_test = train_test_split(paths, labels, test_size=0.2)

In [ ]:
# GAIN API
import cv2
import glob
import torch
import numpy as np
from torch import nn
from PIL import Image
import pretrainedmodels
from matplotlib import pyplot as plt


from config_gain import *
from models import drn, activations
from models.gain import GAINSolver
from models.classifier import ClassifierNet, Classifier

for i in [i / 10 for i in range(-10, 10)]:
# if __name__ == '__main__':
#     i = 0
    backbone = pretrainedmodels.__dict__[model_name]()
    backbone = nn.Sequential(*list(backbone.children())[: -2])
    solver = GAINSolver(backbone, num_classes, in_channels, devices=devices, 
                        activation=activations.HardConcrete(i, temp))
    solver.load_model(
        'saved_models/best_model_GAIN model: se_resnext50_32x4d optimizer: sgd loc: -0.3 temp: 0.1.pt')

    model = pretrainedmodels.__dict__['inceptionresnetv2']()
    net = ClassifierNet(model, 110)
    classifier = Classifier(net, devices=devices)
    classifier.load_model('saved_models/best_model_Classifier model: inceptionresnetv2.pt')

    img = [Image.open(i) for i in glob.iglob('data/dev_data/*') if i.endswith('png')]
    img_np = np.asarray([np.asarray(i) for i in img])

    cls, mask = solver.predict(img)
    mask = mask > 0.5
    img_target = img_np.copy()
    img_target[~mask.repeat(3, axis=-1)] = 0
    num = mask.reshape(mask.shape[0], -1).sum(axis=-1)
    num = num.reshape(mask.shape[0], 1, 1, 1).repeat(3, axis=-1)
    img_mean = img_target.sum(axis=1, keepdims=True).sum(axis=2, keepdims=True) / num
#     img_mean = img_np.mean(axis=1, keepdims=True).mean(axis=2, keepdims=True)
    img_masked = (1 - mask) * img_np + mask * img_mean

    gt = classifier.predict(img).argmax(axis=1)
    pred = solver.predict([Image.fromarray(i) for i in img_masked.astype(np.uint8)])[0].argmax(axis=1)
    true_index = gt == pred
    score = true_index.sum() * 64 / gt.shape[0] + np.sqrt(np.square(img_masked - img_np)[~true_index].mean())
    print('temp: {}, score: {}, acc: {}'.format(i, score, true_index.sum()/110.))

In [ ]:
#coding=utf-8
import os
import csv
import math
import glob
import tqdm
import torch
from PIL import Image

import pretrainedmodels
from config_gba import *
from models.gradient_based_attack import Attack
from models.classifier import ClassifierNet, Classifier


if __name__ == '__main__':
    os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, devices))
    models = [pretrainedmodels.__dict__[name]() for name in classifier_name]
    classifiers = [ClassifierNet(model, num_classes) for model in models]
    for c, p in zip(classifiers, classifier_path):
        c.load_state_dict(torch.load(p))
        
        
    data = {'image_path': [], 'target': [], 'imgs': []}
    with open(os.path.join(data_path, 'dev.csv'), 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            img_path = os.path.join(data_path, row['filename'])
            data['image_path'].append(img_path)
            data['imgs'].append(Image.open(img_path))
            if targeted:
                data['target'].append(int(row['targetedLabel']))
            else:
                data['target'].append(int(row['trueLabel']))
    result = []
    epoch = math.ceil(len(data['imgs']) / batch_size)

    solver = Attack(classifiers[: -1], classifiers[-1: ], 
                    device='cuda', patience=patience, max_iteration=max_iteration)
    for i in tqdm.tqdm(range(epoch), total=epoch):
        torch.cuda.empty_cache()
        batch_x = data['imgs'][i * batch_size: (i + 1) * batch_size]
        batch_y = data['target'][i * batch_size: (i + 1) * batch_size]
        result.extend(solver.predict(batch_x, batch_y, targeted, max_perturbation=max_perturbation, lr=lr))
        

In [ ]:
#coding=utf-8
import os
import csv
import time
import math
import glob
import tqdm
import torch
import numpy as np
from PIL import Image
import torchvision as tv

from models import ddn
import pretrainedmodels
from config_ddn import *
from models.classifier import ClassifierNet, Classifier


if __name__ == '__main__':
    start = time.time()
    os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, devices))
    models = [pretrainedmodels.__dict__[name]() for name in classifier_name]
    classifiers = [ClassifierNet(model, num_classes) for model in models]
    for c, p in zip(classifiers, classifier_path):
        c.load_state_dict(torch.load(p))
        c.eval()
        c = c.cuda()
        
        
    data = {'image_path': [], 'target': [], 'imgs': []}
    with open(os.path.join(data_path, 'dev.csv'), 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            img_path = os.path.join(data_path, row['filename'])
            data['image_path'].append(img_path)
            data['imgs'].append(Image.open(img_path))
            if targeted:
                data['target'].append(int(row['targetedLabel']))
            else:
                data['target'].append(int(row['trueLabel']))
                
    transfrom = tv.transforms.Compose([
        tv.transforms.Resize((224, 224)),
        tv.transforms.ToTensor(),
    #     tv.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    attacker = ddn.DDN(steps=steps, device=torch.device('cuda: 0'))
    epoch = math.ceil(len(data['imgs']) / batch_size)
    result = []
    for i in tqdm.tqdm(range(epoch), total=epoch):
        torch.cuda.empty_cache()
        batch_x = data['imgs'][i * batch_size: (i + 1) * batch_size]
        batch_y = data['target'][i * batch_size: (i + 1) * batch_size]
        
        batch_x = torch.cat([transfrom(i).unsqueeze(dim=0) for i in batch_x], dim=0).cuda()
        batch_y = torch.tensor(batch_y).cuda()
        
        result.append(attacker.attack(classifiers, batch_x, labels=batch_y, targeted=targeted).detach().cpu().data)
    result = torch.cat(result).numpy()
    result = np.transpose(result, (0, 2, 3, 1))
    result = [Image.fromarray(i) for i in (result * 255).astype(np.uint8)]
    print(time.time() - start)